In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Amplitude symmetrization

In [ ]:
import graphviz
import qrules
import sympy as sp
from IPython.display import Math

import ampform
from ampform.dynamics.builder import RelativisticBreitWignerBuilder
from ampform.helicity.decay import perform_combinatorics
from ampform.io import aslatex

Amplitudes for reactions with indistinguishable particles in the final state, such as $D^+ \to \pi^+ \pi^+ \pi^-$, should be symmetrized, meaning that the amplitudes of the different subsystems are indistinguishable but for kinematic variables.

Note that QRules removes transitions that are indistinguishable, since it only considers quantum states, not relativistic kinematics.

In [ ]:
reaction = qrules.generate_transitions(
    initial_state="D+",
    final_state=["pi+", "pi+", "pi-"],
    allowed_intermediate_particles=["rho(770)0"],
)

In [ ]:
src = qrules.io.asdot(reaction, strip_spin=True)
graphviz.Source(src)

Internally, AmpForm's {class}`.HelicityAmplitudeBuilder` permutates these transitions again, so that all kinematically distinguishable subsystems are available.

In [ ]:
assert len(reaction.transitions) == 1
permutated_topologies = perform_combinatorics(reaction.transitions[0])

In [ ]:
src = qrules.io.asdot(permutated_topologies, strip_spin=True)
graphviz.Source(src)

The resulting {class}`.HelicityModel` contains only one amplitude that contains two terms with different kinematic variables:

In [ ]:
bw_builder = RelativisticBreitWignerBuilder()
model_builder = ampform.HelicityAmplitudeBuilder(reaction)
for name in reaction.get_intermediate_particles().names:
    model_builder.dynamics.assign(name, bw_builder)
model = model_builder.formulate()
Math(aslatex(model.amplitudes, terms_per_line=1))

but the parameters of both amplitudes are the same:

In [ ]:
assert len(model.amplitudes) == 1
((symbol, expr),) = model.amplitudes.items()
expr = sp.simplify(expr, doit=False)
Math(aslatex({symbol: expr}))

In [ ]:
Math(aslatex(model.parameter_defaults))